In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [43]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
#train_data.head()

In [45]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
#test_data.head()

In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [46]:
# replacing null values in Age with Mode
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].mode().iloc[0])

In [47]:
# Create Fare class feature
conditions = [
    train_data['Fare'] <= 7.91,
    (train_data['Fare'] > 7.91) & (train_data['Fare'] <= 31.00),
    train_data['Fare'] > 31.00
]
values = [1, 2, 3]
train_data['Fclass'] = np.select(conditions, values, default=0)

In [48]:
#add Fclass to test data too.
conditions = [
    test_data['Fare'] <= 7.91,
    (test_data['Fare'] > 7.91) & (test_data['Fare'] <= 31.00),
    test_data['Fare'] > 31.00
]
values = [1, 2, 3]

test_data['Fclass'] = np.select(conditions, values, default=0)

In [57]:
# Prepare features and target
features = ["Fclass", "Sex", "Age"]
# X = pd.get_dummies(train_data[features], drop_first=True)
# y = train_data["Survived"]
# Ensure features are numeric
X = pd.get_dummies(train_data[features], drop_first=True).astype(float)
y = train_data["Survived"].astype(float)
print(X)

     Fclass   Age  Sex_male
0       1.0  22.0       1.0
1       3.0  38.0       0.0
2       2.0  26.0       0.0
3       3.0  35.0       0.0
4       2.0  35.0       1.0
..      ...   ...       ...
886     2.0  27.0       1.0
887     2.0  19.0       0.0
888     2.0  24.0       0.0
889     2.0  26.0       1.0
890     1.0  32.0       1.0

[891 rows x 3 columns]


In [58]:
from sklearn.model_selection import train_test_split
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Binary Logistic Regression

In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [60]:
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

LogisticRegression(max_iter=200)

In [61]:
# Make predictions on the validation set
y_pred = model.predict(X_val)

In [62]:
# Calculate metrics
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy:.2f}')

conf_matrix = confusion_matrix(y_val, y_pred)
print('Confusion Matrix:\n', conf_matrix)

class_report = classification_report(y_val, y_pred)
print('Classification Report:\n', class_report)


Accuracy: 0.78
Confusion Matrix:
 [[88 17]
 [22 52]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.80      0.84      0.82       105
         1.0       0.75      0.70      0.73        74

    accuracy                           0.78       179
   macro avg       0.78      0.77      0.77       179
weighted avg       0.78      0.78      0.78       179



In [63]:
import statsmodels.api as sm

# Add a constant for the intercept term
X_train_sm = sm.add_constant(X_train)

# Fit the model
logit_model = sm.Logit(y_train, X_train_sm)
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.498954
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  712
Model:                          Logit   Df Residuals:                      708
Method:                           MLE   Df Model:                            3
Date:                Mon, 04 Nov 2024   Pseudo R-squ.:                  0.2466
Time:                        20:54:14   Log-Likelihood:                -355.26
converged:                       True   LL-Null:                       -471.54
Covariance Type:            nonrobust   LLR p-value:                 3.845e-50
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2170      0.363     -0.598      0.550      -0.928       0.494
Fclass         0.6133      0.

In [66]:
# Align X_test with X_train columns, filling any missing columns with 0
X_test = pd.get_dummies(test_data[features], drop_first=True).reindex(columns=X_train.columns, fill_value=0)

# model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
# model.fit(X, y)
# predictions = model.predict(X_test)

# Automatically address null values
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# valid options are 'mean', 'median', 'most_frequent', or 'constant'
imputer = SimpleImputer(strategy='most_frequent')
# model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model = LogisticRegression(max_iter=200)

pipeline = Pipeline(steps=[('imputer', imputer), ('model', model)])
# Train the pipeline
pipeline.fit(X_train, y_train)

# Make predictions on the test data
predictions = pipeline.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})

output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


ChatGPT 4 assisted with above code SimpleImputer, as commented code errored due to NaaN (NULLS).